In [1]:

%pylab inline
import catboost
import pandas as pd
import pickle
import tqdm

import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


# Load data, create target

In [2]:
set_name = 'set1'
path_train_set = 'taxi-idao/data/train/{}.csv'.format(set_name)

data = pd.read_csv(path_train_set)
data.datetime = data.datetime.apply(
    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data = data.sort_values('datetime')
data.head()

,datetime,num_orders
0,2018-03-01 00:00:00,0
1,2018-03-01 00:01:00,0
2,2018-03-01 00:02:00,0
3,2018-03-01 00:03:00,0
4,2018-03-01 00:04:00,1


In [3]:
target_positions = {
    'set1': [10, 30, 45, 60, 75],
    'set2': [5, 10, 15, 20, 25],
    'set3': [5, 7, 9, 11, 13]
}[set_name]



HOUR_IN_MINUTES = 60
DAY_IN_MINUTES = 24 * HOUR_IN_MINUTES
WEEK_IN_MINUTES = 7 * DAY_IN_MINUTES

MAX_TIME = DAY_IN_MINUTES


samples = {
    'datetime': [],
    'history': []}

for position in target_positions:
    samples['target_{}'.format(position)] = []
    
num_orders = data.num_orders.values



# start after 2 weeks because of history
# finish earlier because of target calculation
for i in range(2 * WEEK_IN_MINUTES,
               len(num_orders) - 2 * DAY_IN_MINUTES):
    
    samples['datetime'].append(data.datetime[i])
    samples['history'].append(num_orders[i-2*WEEK_IN_MINUTES:i])
    
    # cumsum not for all array because of time economy
    cumsum_num_orders = num_orders[i+1:i+1+2*DAY_IN_MINUTES].cumsum()
    for position in target_positions:
        orders_by_positions = np.where(cumsum_num_orders >= position)[0]
        if len(orders_by_positions):
            time = orders_by_positions[0] + 1
        else:
            # if no orders in last days
            time = MAX_TIME
        samples['target_{}'.format(position)].append(time)

In [4]:
df = pd.DataFrame.from_dict(samples)
df.head()

,datetime,history,target_10,target_30,target_45,target_60,target_75
0,2018-03-15 00:00:00,"[0, 0, 0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, ...",5,18,28,32,42
1,2018-03-15 00:01:00,"[0, 0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, ...",5,19,27,32,42
2,2018-03-15 00:02:00,"[0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, 2, ...",7,20,27,33,43
3,2018-03-15 00:03:00,"[0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, 2, 3, ...",7,21,26,35,42
4,2018-03-15 00:04:00,"[1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, 2, 3, 1, ...",7,20,26,35,42


## Base features

In [5]:
df['weekday'] = df.datetime.apply(lambda x: x.weekday())
df['hour'] = df.datetime.apply(lambda x: x.hour)
df['minute'] = df.datetime.apply(lambda x: x.minute)


SHIFTS = [
    HOUR_IN_MINUTES // 4,
    HOUR_IN_MINUTES // 2,
    HOUR_IN_MINUTES,
    DAY_IN_MINUTES,
    DAY_IN_MINUTES * 2,
    WEEK_IN_MINUTES,
    WEEK_IN_MINUTES * 2]
WINDOWS = [
    HOUR_IN_MINUTES // 4 - 1,
    HOUR_IN_MINUTES // 2 - 1,
    HOUR_IN_MINUTES - 1,
    DAY_IN_MINUTES - 1,
    DAY_IN_MINUTES * 2 - 1,
    WEEK_IN_MINUTES - 1,
    WEEK_IN_MINUTES * 2 - 1]


all_features_names = ['weekday', 'hour', 'minute']
for shift in SHIFTS:
    for window in WINDOWS:
        if window > shift:
            continue
        all_features_names.append('num_orders_{}_{}'.format(shift, window))
        df['num_orders_{}_{}'.format(shift, window)] = \
            df.history.apply(lambda x: x[-shift : -shift + window].sum())

## Percentiles

In [6]:
SHIFTS_PERCENTILE = [WEEK_IN_MINUTES, WEEK_IN_MINUTES * 2, DAY_IN_MINUTES]
WINDOWS_PERCENTILE = {10: [3, 5, 6, 7, 8, 10, 14, 32],
                      30: [10, 15, 17, 20, 23, 28, 37, 76],
                      45: [15, 22, 26, 30, 34, 42, 54, 102],
                      60: [21, 30, 35, 40, 45, 55, 69, 125],
                      75: [26, 38, 44, 50, 56, 68, 85, 148]}


shifts_names = ['week_ago', 'two_weeks_ago', 'day_ago']
windows_names = ['5%', '20%', '30%', '40%', '50%', '65%', '80%', '95%']

In [7]:
### part 1

In [8]:
df_history = df.history.values

for target in tqdm.tqdm(target_positions):
    for i, shift in enumerate(SHIFTS_PERCENTILE):
        for j, window in enumerate(WINDOWS_PERCENTILE[target]):
            df['num_orders_target_{}_{}_{}'.format(target, shifts_names[i], windows_names[j])] = list(map(lambda x: x[-shift: -shift + window].sum(), df_history))

100%|██████████| 5/5 [02:07<00:00, 25.42s/it]


In [9]:
### part 2

In [10]:
for target in tqdm.tqdm(target_positions):
    for j, window in enumerate(WINDOWS_PERCENTILE[target]):
        col_name = 'num_orders_target_{}_previous_{}_perc'.format(target, windows_names[j])
        df[col_name] = list(map(lambda x: x[-window:].sum(), df_history))

100%|██████████| 5/5 [00:44<00:00,  8.98s/it]


In [11]:
df.head(2)

,datetime,history,target_10,target_30,target_45,target_60,target_75,weekday,hour,minute,...,num_orders_target_60_previous_80%_perc,num_orders_target_60_previous_95%_perc,num_orders_target_75_previous_5%_perc,num_orders_target_75_previous_20%_perc,num_orders_target_75_previous_30%_perc,num_orders_target_75_previous_40%_perc,num_orders_target_75_previous_50%_perc,num_orders_target_75_previous_65%_perc,num_orders_target_75_previous_80%_perc,num_orders_target_75_previous_95%_perc
0,2018-03-15 00:00:00,"[0, 0, 0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, ...",5,18,28,32,42,3,0,0,...,72,150,45,51,53,53,63,71,92,172
1,2018-03-15 00:01:00,"[0, 0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, ...",5,19,27,32,42,3,0,1,...,75,154,47,55,57,57,66,73,94,176


## Creating lists

In [36]:
ALL_FEATURES = df.columns[7:]

In [37]:
len(ALL_FEATURES)

192

In [38]:
ALL_FEATURES[:31]

Index(['weekday', 'hour', 'minute', 'num_orders_15_14', 'num_orders_30_14',
       'num_orders_30_29', 'num_orders_60_14', 'num_orders_60_29',
       'num_orders_60_59', 'num_orders_1440_14', 'num_orders_1440_29',
       'num_orders_1440_59', 'num_orders_1440_1439', 'num_orders_2880_14',
       'num_orders_2880_29', 'num_orders_2880_59', 'num_orders_2880_1439',
       'num_orders_2880_2879', 'num_orders_10080_14', 'num_orders_10080_29',
       'num_orders_10080_59', 'num_orders_10080_1439', 'num_orders_10080_2879',
       'num_orders_10080_10079', 'num_orders_20160_14', 'num_orders_20160_29',
       'num_orders_20160_59', 'num_orders_20160_1439', 'num_orders_20160_2879',
       'num_orders_20160_10079', 'num_orders_20160_20159'],
      dtype='object')

In [39]:
TARGET_FEATURES_POSITIONS = {}
TARGET_FEATURES_POSITIONS[10] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[30] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[45] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[60] = [i for i in range(31)]
TARGET_FEATURES_POSITIONS[75] = [i for i in range(31)]

In [40]:
### target 10
TARGET_FEATURES_POSITIONS[10] = TARGET_FEATURES_POSITIONS[10] + [i for i in range(31,31+24)]
TARGET_FEATURES_POSITIONS[30] = TARGET_FEATURES_POSITIONS[30] + [i for i in range(31+24,31+24*2)]
TARGET_FEATURES_POSITIONS[45] = TARGET_FEATURES_POSITIONS[45] + [i for i in range(31+24*2,31+24*3)]
TARGET_FEATURES_POSITIONS[60] = TARGET_FEATURES_POSITIONS[60] + [i for i in range(31+24*3,31+24*4)]
TARGET_FEATURES_POSITIONS[75] = TARGET_FEATURES_POSITIONS[75] + [i for i in range(31+24*4,31+24*5)]




TARGET_FEATURES_POSITIONS[10] = TARGET_FEATURES_POSITIONS[10] + [i for i in range(151,151+8)]
TARGET_FEATURES_POSITIONS[30] = TARGET_FEATURES_POSITIONS[30] + [i for i in range(151+8,151+8*2)]
TARGET_FEATURES_POSITIONS[45] = TARGET_FEATURES_POSITIONS[45] + [i for i in range(151+8*2,151+8*3)]
TARGET_FEATURES_POSITIONS[60] = TARGET_FEATURES_POSITIONS[60] + [i for i in range(151+8*3,151+8*4)]
TARGET_FEATURES_POSITIONS[75] = TARGET_FEATURES_POSITIONS[75] + [i for i in range(151+8*4,151+8*5)]

# Split data

In [60]:
df.datetime.min(), df.datetime.max()
df_train = df.loc[df.datetime <= df.datetime.max() - datetime.timedelta(days=28)]
df_test = df.loc[(df.datetime > df.datetime.max() - datetime.timedelta(days=14))]

target_cols = ['target_{}'.format(position) for position in target_positions]

y_train = df_train[target_cols]
y_test = df_test[target_cols]

df_train = df_train.drop(['datetime', 'history'] + target_cols, axis=1)
df_test = df_test.drop(['datetime', 'history'] + target_cols, axis=1)



def sMAPE(y_true, y_predict, shift=0):
    return 2 * np.mean(
        np.abs(y_true - y_predict) /
        (np.abs(y_true) + np.abs(y_predict) + shift))


model_to_save = {
    'models': {}
}

# Train for validation

In [70]:
### all features
for position in target_positions:
    model = catboost.CatBoostRegressor(
        iterations=600, learning_rate=1.0, loss_function='MAE')
    model.fit(
        X=df_train[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]],
        y=y_train['target_{}'.format(position)],
        use_best_model=True,
        eval_set=(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]], y_test['target_{}'.format(position)]),
        verbose=True)
    y_predict = model.predict(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]])
    
    print('target_{}'.format(position))
    print('stupid:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_train['target_{}'.format(position)].median())))
    print('model:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_predict)))
    print()
    
    model_to_save['models'][position] = model

0:	learn: 12.0297470	test: 5.8899454	best: 5.8899454 (0)	total: 98ms	remaining: 58.7s
1:	learn: 11.5297619	test: 5.3899603	best: 5.3899603 (1)	total: 130ms	remaining: 38.9s
2:	learn: 11.0298958	test: 4.8947367	best: 4.8947367 (2)	total: 158ms	remaining: 31.3s
3:	learn: 10.5302559	test: 4.4161102	best: 4.4161102 (3)	total: 227ms	remaining: 33.8s
4:	learn: 10.0381349	test: 3.9821172	best: 3.9821172 (4)	total: 293ms	remaining: 34.9s
5:	learn: 9.5527587	test: 3.5807769	best: 3.5807769 (5)	total: 409ms	remaining: 40.5s
6:	learn: 9.0996207	test: 3.2758403	best: 3.2758403 (6)	total: 486ms	remaining: 41.2s
7:	learn: 8.6741753	test: 3.0340998	best: 3.0340998 (7)	total: 560ms	remaining: 41.4s
8:	learn: 8.2911527	test: 2.8411147	best: 2.8411147 (8)	total: 637ms	remaining: 41.8s
9:	learn: 7.9566836	test: 2.7139101	best: 2.7139101 (9)	total: 718ms	remaining: 42.4s
10:	learn: 7.6522497	test: 2.6047846	best: 2.6047846 (10)	total: 784ms	remaining: 42s
11:	learn: 7.3984976	test: 2.5271071	best: 2.52710

96:	learn: 4.4679344	test: 2.0054770	best: 2.0047036 (93)	total: 9.1s	remaining: 47.2s
97:	learn: 4.4601747	test: 2.0053197	best: 2.0047036 (93)	total: 9.18s	remaining: 47s
98:	learn: 4.4527803	test: 2.0051399	best: 2.0047036 (93)	total: 9.25s	remaining: 46.8s
99:	learn: 4.4455263	test: 2.0048948	best: 2.0047036 (93)	total: 9.43s	remaining: 47.2s
100:	learn: 4.4377462	test: 2.0052370	best: 2.0047036 (93)	total: 9.56s	remaining: 47.2s
101:	learn: 4.4305774	test: 2.0050458	best: 2.0047036 (93)	total: 9.7s	remaining: 47.3s
102:	learn: 4.4235021	test: 2.0049312	best: 2.0047036 (93)	total: 9.8s	remaining: 47.3s
103:	learn: 4.4162588	test: 2.0045679	best: 2.0045679 (103)	total: 9.91s	remaining: 47.2s
104:	learn: 4.4095377	test: 2.0047536	best: 2.0045679 (103)	total: 10s	remaining: 47.3s
105:	learn: 4.4027849	test: 2.0045454	best: 2.0045454 (105)	total: 10.1s	remaining: 47.2s
106:	learn: 4.3959356	test: 2.0043257	best: 2.0043257 (106)	total: 10.2s	remaining: 47.1s
107:	learn: 4.3889127	test: 

190:	learn: 3.9989281	test: 2.0023303	best: 2.0023303 (190)	total: 18.6s	remaining: 39.9s
191:	learn: 3.9958290	test: 2.0024097	best: 2.0023303 (190)	total: 18.7s	remaining: 39.8s
192:	learn: 3.9928463	test: 2.0024340	best: 2.0023303 (190)	total: 18.8s	remaining: 39.7s
193:	learn: 3.9903169	test: 2.0025231	best: 2.0023303 (190)	total: 18.9s	remaining: 39.6s
194:	learn: 3.9867593	test: 2.0024366	best: 2.0023303 (190)	total: 19s	remaining: 39.4s
195:	learn: 3.9840312	test: 2.0026333	best: 2.0023303 (190)	total: 19.1s	remaining: 39.3s
196:	learn: 3.9811923	test: 2.0026581	best: 2.0023303 (190)	total: 19.2s	remaining: 39.2s
197:	learn: 3.9784296	test: 2.0026353	best: 2.0023303 (190)	total: 19.3s	remaining: 39.1s
198:	learn: 3.9752705	test: 2.0026640	best: 2.0023303 (190)	total: 19.4s	remaining: 39.1s
199:	learn: 3.9726106	test: 2.0028270	best: 2.0023303 (190)	total: 19.5s	remaining: 38.9s
200:	learn: 3.9697109	test: 2.0027021	best: 2.0023303 (190)	total: 19.6s	remaining: 38.9s
201:	learn: 

284:	learn: 3.7915330	test: 1.9902489	best: 1.9902489 (284)	total: 27.2s	remaining: 30.1s
285:	learn: 3.7894804	test: 1.9903649	best: 1.9902489 (284)	total: 27.3s	remaining: 30s
286:	learn: 3.7877786	test: 1.9904238	best: 1.9902489 (284)	total: 27.4s	remaining: 29.9s
287:	learn: 3.7861105	test: 1.9901119	best: 1.9901119 (287)	total: 27.5s	remaining: 29.8s
288:	learn: 3.7840367	test: 1.9902121	best: 1.9901119 (287)	total: 27.5s	remaining: 29.6s
289:	learn: 3.7821242	test: 1.9900924	best: 1.9900924 (289)	total: 27.6s	remaining: 29.5s
290:	learn: 3.7801946	test: 1.9901144	best: 1.9900924 (289)	total: 27.7s	remaining: 29.4s
291:	learn: 3.7785328	test: 1.9901224	best: 1.9900924 (289)	total: 27.8s	remaining: 29.3s
292:	learn: 3.7768258	test: 1.9901018	best: 1.9900924 (289)	total: 27.9s	remaining: 29.2s
293:	learn: 3.7751367	test: 1.9901205	best: 1.9900924 (289)	total: 27.9s	remaining: 29.1s
294:	learn: 3.7725487	test: 1.9902655	best: 1.9900924 (289)	total: 28s	remaining: 29s
295:	learn: 3.77

377:	learn: 3.6500688	test: 2.0241137	best: 1.9887181 (298)	total: 34.6s	remaining: 20.3s
378:	learn: 3.6485501	test: 2.0237306	best: 1.9887181 (298)	total: 34.6s	remaining: 20.2s
379:	learn: 3.6473697	test: 2.0240202	best: 1.9887181 (298)	total: 34.7s	remaining: 20.1s
380:	learn: 3.6463142	test: 2.0251338	best: 1.9887181 (298)	total: 34.8s	remaining: 20s
381:	learn: 3.6444978	test: 2.0251807	best: 1.9887181 (298)	total: 34.9s	remaining: 19.9s
382:	learn: 3.6428326	test: 2.0254367	best: 1.9887181 (298)	total: 34.9s	remaining: 19.8s
383:	learn: 3.6420826	test: 2.0252737	best: 1.9887181 (298)	total: 35s	remaining: 19.7s
384:	learn: 3.6407527	test: 2.0259474	best: 1.9887181 (298)	total: 35.1s	remaining: 19.6s
385:	learn: 3.6386372	test: 2.0259709	best: 1.9887181 (298)	total: 35.2s	remaining: 19.5s
386:	learn: 3.6369338	test: 2.0259487	best: 1.9887181 (298)	total: 35.3s	remaining: 19.4s
387:	learn: 3.6363817	test: 2.0262334	best: 1.9887181 (298)	total: 35.3s	remaining: 19.3s
388:	learn: 3.

...

200:	learn: 16.9748217	test: 8.3938235	best: 8.3938235 (200)	total: 14.5s	remaining: 28.9s
201:	learn: 16.9272400	test: 8.3896240	best: 8.3896240 (201)	total: 14.6s	remaining: 28.8s
202:	learn: 16.8804665	test: 8.3881539	best: 8.3881539 (202)	total: 14.7s	remaining: 28.7s
203:	learn: 16.8346469	test: 8.3788920	best: 8.3788920 (203)	total: 14.8s	remaining: 28.6s
204:	learn: 16.7897450	test: 8.3762041	best: 8.3762041 (204)	total: 14.8s	remaining: 28.6s
205:	learn: 16.7453857	test: 8.3742181	best: 8.3742181 (205)	total: 14.9s	remaining: 28.5s
206:	learn: 16.7016973	test: 8.3745963	best: 8.3742181 (205)	total: 15s	remaining: 28.5s
207:	learn: 16.6576568	test: 8.3639347	best: 8.3639347 (207)	total: 15.1s	remaining: 28.4s
208:	learn: 16.6140669	test: 8.3567515	best: 8.3567515 (208)	total: 15.1s	remaining: 28.3s
209:	learn: 16.5715950	test: 8.3459872	best: 8.3459872 (209)	total: 15.2s	remaining: 28.2s
210:	learn: 16.5286102	test: 8.3448678	best: 8.3448678 (210)	total: 15.3s	remaining: 28.2s
2

291:	learn: 13.9169728	test: 8.2245312	best: 8.2209418 (286)	total: 21.6s	remaining: 22.7s
292:	learn: 13.8932075	test: 8.2235190	best: 8.2209418 (286)	total: 21.6s	remaining: 22.7s
293:	learn: 13.8697508	test: 8.2246130	best: 8.2209418 (286)	total: 21.7s	remaining: 22.6s
294:	learn: 13.8466584	test: 8.2260143	best: 8.2209418 (286)	total: 21.8s	remaining: 22.5s
295:	learn: 13.8235764	test: 8.2269319	best: 8.2209418 (286)	total: 21.9s	remaining: 22.5s
296:	learn: 13.8008000	test: 8.2271528	best: 8.2209418 (286)	total: 21.9s	remaining: 22.4s
297:	learn: 13.7770064	test: 8.2286842	best: 8.2209418 (286)	total: 22s	remaining: 22.3s
298:	learn: 13.7539819	test: 8.2276887	best: 8.2209418 (286)	total: 22.1s	remaining: 22.2s
299:	learn: 13.7315754	test: 8.2293110	best: 8.2209418 (286)	total: 22.2s	remaining: 22.2s
300:	learn: 13.7080987	test: 8.2283385	best: 8.2209418 (286)	total: 22.2s	remaining: 22.1s
301:	learn: 13.6846819	test: 8.2292184	best: 8.2209418 (286)	total: 22.3s	remaining: 22s
302

382:	learn: 12.3245447	test: 8.2238695	best: 8.2048535 (350)	total: 28.5s	remaining: 16.2s
383:	learn: 12.3124313	test: 8.2228329	best: 8.2048535 (350)	total: 28.6s	remaining: 16.1s
384:	learn: 12.3006007	test: 8.2230067	best: 8.2048535 (350)	total: 28.7s	remaining: 16s
385:	learn: 12.2895054	test: 8.2219507	best: 8.2048535 (350)	total: 28.7s	remaining: 15.9s
386:	learn: 12.2786217	test: 8.2235454	best: 8.2048535 (350)	total: 28.8s	remaining: 15.9s
387:	learn: 12.2662717	test: 8.2229620	best: 8.2048535 (350)	total: 28.9s	remaining: 15.8s
388:	learn: 12.2554212	test: 8.2237707	best: 8.2048535 (350)	total: 29s	remaining: 15.7s
389:	learn: 12.2439091	test: 8.2251642	best: 8.2048535 (350)	total: 29s	remaining: 15.6s
390:	learn: 12.2329671	test: 8.2269779	best: 8.2048535 (350)	total: 29.1s	remaining: 15.6s
391:	learn: 12.2228665	test: 8.2248325	best: 8.2048535 (350)	total: 29.2s	remaining: 15.5s
392:	learn: 12.2115976	test: 8.2236814	best: 8.2048535 (350)	total: 29.3s	remaining: 15.4s
393:	

474:	learn: 11.5035649	test: 8.1935406	best: 8.1935406 (474)	total: 35.6s	remaining: 9.37s
475:	learn: 11.4986798	test: 8.1899143	best: 8.1899143 (475)	total: 35.7s	remaining: 9.3s
476:	learn: 11.4922371	test: 8.1898463	best: 8.1898463 (476)	total: 35.8s	remaining: 9.22s
477:	learn: 11.4867371	test: 8.1901014	best: 8.1898463 (476)	total: 35.9s	remaining: 9.15s
478:	learn: 11.4809977	test: 8.1871219	best: 8.1871219 (478)	total: 35.9s	remaining: 9.08s
479:	learn: 11.4754678	test: 8.1876267	best: 8.1871219 (478)	total: 36s	remaining: 9s
480:	learn: 11.4693246	test: 8.1872128	best: 8.1871219 (478)	total: 36.1s	remaining: 8.92s
481:	learn: 11.4635460	test: 8.1885016	best: 8.1871219 (478)	total: 36.1s	remaining: 8.85s
482:	learn: 11.4549981	test: 8.1881859	best: 8.1871219 (478)	total: 36.2s	remaining: 8.77s
483:	learn: 11.4488880	test: 8.1880942	best: 8.1871219 (478)	total: 36.3s	remaining: 8.7s
484:	learn: 11.4408658	test: 8.1883538	best: 8.1871219 (478)	total: 36.4s	remaining: 8.62s
485:	l

565:	learn: 11.0242518	test: 8.2044559	best: 8.1555001 (499)	total: 42.5s	remaining: 2.56s
566:	learn: 11.0204352	test: 8.2049143	best: 8.1555001 (499)	total: 42.6s	remaining: 2.48s
567:	learn: 11.0145927	test: 8.2050420	best: 8.1555001 (499)	total: 42.7s	remaining: 2.4s
568:	learn: 11.0099846	test: 8.2336899	best: 8.1555001 (499)	total: 42.8s	remaining: 2.33s
569:	learn: 11.0050928	test: 8.2327158	best: 8.1555001 (499)	total: 42.9s	remaining: 2.25s
570:	learn: 11.0004644	test: 8.2326142	best: 8.1555001 (499)	total: 42.9s	remaining: 2.18s
571:	learn: 10.9970296	test: 8.2319685	best: 8.1555001 (499)	total: 43s	remaining: 2.1s
572:	learn: 10.9925476	test: 8.2334070	best: 8.1555001 (499)	total: 43.1s	remaining: 2.03s
573:	learn: 10.9880925	test: 8.2335681	best: 8.1555001 (499)	total: 43.1s	remaining: 1.95s
574:	learn: 10.9829411	test: 8.2346888	best: 8.1555001 (499)	total: 43.2s	remaining: 1.88s
575:	learn: 10.9768465	test: 8.2343567	best: 8.1555001 (499)	total: 43.3s	remaining: 1.8s
576:

# Full learning

In [72]:
df_full_train = df.drop(['datetime', 'history'] + target_cols, axis=1)
y_full_train = df[target_cols]

In [87]:
dict_iterations = {10: 370, 
                   30: 441, 
                   45: 492, 
                   60: 509, 
                   75: 546}

In [89]:
for position in target_positions:
    model = catboost.CatBoostRegressor(
        iterations=dict_iterations[position], learning_rate=1.0, loss_function='MAE')
    model.fit(
        X=df_full_train[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]],
        y=y_full_train['target_{}'.format(position)],
        use_best_model=False,
        eval_set=(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]], y_test['target_{}'.format(position)]),
        verbose=True)
    y_predict = model.predict(df_test[ALL_FEATURES[TARGET_FEATURES_POSITIONS[position]]])
    
    print('target_{}'.format(position))
    print('stupid:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_train['target_{}'.format(position)].median())))
    print('model:\t{}'.format(sMAPE(
        y_test['target_{}'.format(position)],
        y_predict)))
    print()
    
    model_to_save['models'][position] = model

0:	learn: 11.1245288	test: 5.8899430	best: 5.8899430 (0)	total: 47.6ms	remaining: 17.6s
1:	learn: 10.6245412	test: 5.3899554	best: 5.3899554 (1)	total: 77.8ms	remaining: 14.3s
2:	learn: 10.1252025	test: 4.8947293	best: 4.8947293 (2)	total: 108ms	remaining: 13.2s
3:	learn: 9.6263253	test: 4.4014507	best: 4.4014507 (3)	total: 186ms	remaining: 17s
4:	learn: 9.1416607	test: 3.9711261	best: 3.9711261 (4)	total: 269ms	remaining: 19.6s
5:	learn: 8.6674615	test: 3.5756120	best: 3.5756120 (5)	total: 377ms	remaining: 22.8s
6:	learn: 8.2339548	test: 3.2703448	best: 3.2703448 (6)	total: 461ms	remaining: 23.9s
7:	learn: 7.8350232	test: 3.0321397	best: 3.0321397 (7)	total: 559ms	remaining: 25.3s
8:	learn: 7.4778592	test: 2.8397451	best: 2.8397451 (8)	total: 649ms	remaining: 26s
9:	learn: 7.1734710	test: 2.7091685	best: 2.7091685 (9)	total: 728ms	remaining: 26.2s
10:	learn: 6.8965541	test: 2.5966098	best: 2.5966098 (10)	total: 822ms	remaining: 26.8s
11:	learn: 6.6681694	test: 2.5166668	best: 2.516666

96:	learn: 4.1008241	test: 1.9535885	best: 1.9535885 (96)	total: 8.83s	remaining: 24.9s
97:	learn: 4.0945560	test: 1.9535340	best: 1.9535340 (97)	total: 8.95s	remaining: 24.8s
98:	learn: 4.0882642	test: 1.9532607	best: 1.9532607 (98)	total: 9.04s	remaining: 24.8s
99:	learn: 4.0819522	test: 1.9528477	best: 1.9528477 (99)	total: 9.14s	remaining: 24.7s
100:	learn: 4.0757554	test: 1.9526001	best: 1.9526001 (100)	total: 9.22s	remaining: 24.6s
101:	learn: 4.0697709	test: 1.9527527	best: 1.9526001 (100)	total: 9.32s	remaining: 24.5s
102:	learn: 4.0638566	test: 1.9527316	best: 1.9526001 (100)	total: 9.4s	remaining: 24.4s
103:	learn: 4.0579541	test: 1.9526683	best: 1.9526001 (100)	total: 9.48s	remaining: 24.3s
104:	learn: 4.0518808	test: 1.9524615	best: 1.9524615 (104)	total: 9.59s	remaining: 24.2s
105:	learn: 4.0460135	test: 1.9523772	best: 1.9523772 (105)	total: 9.67s	remaining: 24.1s
106:	learn: 4.0402092	test: 1.9523358	best: 1.9523358 (106)	total: 9.75s	remaining: 24s
107:	learn: 4.0345330

188:	learn: 3.7131161	test: 1.9490115	best: 1.9490115 (188)	total: 17.1s	remaining: 16.4s
189:	learn: 3.7104457	test: 1.9488778	best: 1.9488778 (189)	total: 17.2s	remaining: 16.3s
190:	learn: 3.7085420	test: 1.9479912	best: 1.9479912 (190)	total: 17.3s	remaining: 16.2s
191:	learn: 3.7056519	test: 1.9480046	best: 1.9479912 (190)	total: 17.4s	remaining: 16.1s
192:	learn: 3.7034893	test: 1.9480983	best: 1.9479912 (190)	total: 17.5s	remaining: 16.1s
193:	learn: 3.7010366	test: 1.9480745	best: 1.9479912 (190)	total: 17.6s	remaining: 16s
194:	learn: 3.6986564	test: 1.9481181	best: 1.9479912 (190)	total: 17.7s	remaining: 15.9s
195:	learn: 3.6960167	test: 1.9480986	best: 1.9479912 (190)	total: 17.8s	remaining: 15.8s
196:	learn: 3.6937741	test: 1.9481270	best: 1.9479912 (190)	total: 18s	remaining: 15.8s
197:	learn: 3.6914108	test: 1.9478199	best: 1.9478199 (197)	total: 18.1s	remaining: 15.7s
198:	learn: 3.6891495	test: 1.9478810	best: 1.9478199 (197)	total: 18.3s	remaining: 15.7s
199:	learn: 3.

281:	learn: 3.5401394	test: 1.9349536	best: 1.9349536 (281)	total: 26.8s	remaining: 8.36s
282:	learn: 3.5390539	test: 1.9349728	best: 1.9349536 (281)	total: 26.9s	remaining: 8.27s
283:	learn: 3.5381138	test: 1.9336529	best: 1.9336529 (283)	total: 27s	remaining: 8.17s
284:	learn: 3.5367271	test: 1.9335136	best: 1.9335136 (284)	total: 27.1s	remaining: 8.08s
285:	learn: 3.5357857	test: 1.9318533	best: 1.9318533 (285)	total: 27.2s	remaining: 7.98s
286:	learn: 3.5340443	test: 1.9318036	best: 1.9318036 (286)	total: 27.3s	remaining: 7.88s
287:	learn: 3.5327059	test: 1.9318306	best: 1.9318036 (286)	total: 27.4s	remaining: 7.79s
288:	learn: 3.5308274	test: 1.9317862	best: 1.9317862 (288)	total: 27.4s	remaining: 7.69s
289:	learn: 3.5297990	test: 1.9317688	best: 1.9317688 (289)	total: 27.5s	remaining: 7.59s
290:	learn: 3.5285952	test: 1.9317827	best: 1.9317688 (289)	total: 27.6s	remaining: 7.5s
291:	learn: 3.5272616	test: 1.9311461	best: 1.9311461 (291)	total: 27.7s	remaining: 7.4s
292:	learn: 3.

7:	learn: 25.6774672	test: 13.1163252	best: 13.1163252 (7)	total: 314ms	remaining: 17s
8:	learn: 25.1780443	test: 12.6208826	best: 12.6208826 (8)	total: 391ms	remaining: 18.8s
9:	learn: 24.6788784	test: 12.1262989	best: 12.1262989 (9)	total: 469ms	remaining: 20.2s
10:	learn: 24.1808202	test: 11.6402632	best: 11.6402632 (10)	total: 566ms	remaining: 22.1s
11:	learn: 23.6837011	test: 11.1592974	best: 11.1592974 (11)	total: 645ms	remaining: 23s
12:	learn: 23.1894103	test: 10.6934120	best: 10.6934120 (12)	total: 722ms	remaining: 23.8s
13:	learn: 22.6977422	test: 10.2436908	best: 10.2436908 (13)	total: 859ms	remaining: 26.2s
14:	learn: 22.2104033	test: 9.8096276	best: 9.8096276 (14)	total: 945ms	remaining: 26.8s
15:	learn: 21.7282001	test: 9.3948811	best: 9.3948811 (15)	total: 1.02s	remaining: 27.2s
16:	learn: 21.2524610	test: 9.0029043	best: 9.0029043 (16)	total: 1.11s	remaining: 27.8s
17:	learn: 20.7846998	test: 8.6360041	best: 8.6360041 (17)	total: 1.2s	remaining: 28.2s
18:	learn: 20.3252

...

132:	learn: 20.1280729	test: 8.5399033	best: 8.5399033 (132)	total: 14.6s	remaining: 45.3s
133:	learn: 20.0215400	test: 8.5124833	best: 8.5124833 (133)	total: 14.7s	remaining: 45.2s
134:	learn: 19.9173416	test: 8.4864661	best: 8.4864661 (134)	total: 14.8s	remaining: 45.1s
135:	learn: 19.8130774	test: 8.4584462	best: 8.4584462 (135)	total: 14.9s	remaining: 45.1s
136:	learn: 19.7109432	test: 8.4315163	best: 8.4315163 (136)	total: 15.1s	remaining: 45s
137:	learn: 19.6115706	test: 8.4080307	best: 8.4080307 (137)	total: 15.2s	remaining: 44.9s
138:	learn: 19.5149330	test: 8.3866518	best: 8.3866518 (138)	total: 15.3s	remaining: 44.7s
139:	learn: 19.4187286	test: 8.3657103	best: 8.3657103 (139)	total: 15.4s	remaining: 44.7s
140:	learn: 19.3250364	test: 8.3424645	best: 8.3424645 (140)	total: 15.5s	remaining: 44.6s
141:	learn: 19.2317261	test: 8.3169965	best: 8.3169965 (141)	total: 15.7s	remaining: 44.6s
142:	learn: 19.1397790	test: 8.2931533	best: 8.2931533 (142)	total: 15.8s	remaining: 44.5s
1

224:	learn: 14.7742478	test: 7.3481217	best: 7.3481217 (224)	total: 25.8s	remaining: 36.8s
225:	learn: 14.7415101	test: 7.3452217	best: 7.3452217 (225)	total: 25.9s	remaining: 36.7s
226:	learn: 14.7081226	test: 7.3414177	best: 7.3414177 (226)	total: 26s	remaining: 36.6s
227:	learn: 14.6759848	test: 7.3384111	best: 7.3384111 (227)	total: 26.1s	remaining: 36.5s
228:	learn: 14.6438671	test: 7.3354818	best: 7.3354818 (228)	total: 26.3s	remaining: 36.4s
229:	learn: 14.6127440	test: 7.3281603	best: 7.3281603 (229)	total: 26.4s	remaining: 36.3s
230:	learn: 14.5815953	test: 7.3234603	best: 7.3234603 (230)	total: 26.6s	remaining: 36.2s
231:	learn: 14.5498914	test: 7.3208811	best: 7.3208811 (231)	total: 26.7s	remaining: 36.1s
232:	learn: 14.5183855	test: 7.3176150	best: 7.3176150 (232)	total: 26.8s	remaining: 36s
233:	learn: 14.4882638	test: 7.3095634	best: 7.3095634 (233)	total: 27s	remaining: 35.9s
234:	learn: 14.4573409	test: 7.3064202	best: 7.3064202 (234)	total: 27.1s	remaining: 35.8s
235:	

315:	learn: 12.5876196	test: 7.1715447	best: 7.1709406 (312)	total: 38.5s	remaining: 28s
316:	learn: 12.5698879	test: 7.1717293	best: 7.1709406 (312)	total: 38.6s	remaining: 27.9s
317:	learn: 12.5534316	test: 7.1720659	best: 7.1709406 (312)	total: 38.7s	remaining: 27.8s
318:	learn: 12.5371622	test: 7.1722184	best: 7.1709406 (312)	total: 38.9s	remaining: 27.7s
319:	learn: 12.5215493	test: 7.1717998	best: 7.1709406 (312)	total: 39s	remaining: 27.6s
320:	learn: 12.5060240	test: 7.1718817	best: 7.1709406 (312)	total: 39.2s	remaining: 27.4s
321:	learn: 12.4894721	test: 7.1715786	best: 7.1709406 (312)	total: 39.4s	remaining: 27.4s
322:	learn: 12.4730679	test: 7.1719036	best: 7.1709406 (312)	total: 39.5s	remaining: 27.3s
323:	learn: 12.4569562	test: 7.1723693	best: 7.1709406 (312)	total: 39.6s	remaining: 27.2s
324:	learn: 12.4415465	test: 7.1720855	best: 7.1709406 (312)	total: 39.8s	remaining: 27s
325:	learn: 12.4256985	test: 7.1719011	best: 7.1709406 (312)	total: 39.9s	remaining: 26.9s
326:	

406:	learn: 11.4654844	test: 7.1513678	best: 7.1507738 (405)	total: 52.3s	remaining: 17.8s
407:	learn: 11.4564875	test: 7.1513208	best: 7.1507738 (405)	total: 52.4s	remaining: 17.7s
408:	learn: 11.4478531	test: 7.1515175	best: 7.1507738 (405)	total: 52.5s	remaining: 17.6s
409:	learn: 11.4400653	test: 7.1519601	best: 7.1507738 (405)	total: 52.6s	remaining: 17.5s
410:	learn: 11.4311744	test: 7.1520129	best: 7.1507738 (405)	total: 52.8s	remaining: 17.3s
411:	learn: 11.4219751	test: 7.1522353	best: 7.1507738 (405)	total: 52.9s	remaining: 17.2s
412:	learn: 11.4141465	test: 7.1517639	best: 7.1507738 (405)	total: 53s	remaining: 17.1s
413:	learn: 11.4059924	test: 7.1514977	best: 7.1507738 (405)	total: 53.1s	remaining: 16.9s
414:	learn: 11.3968637	test: 7.1519863	best: 7.1507738 (405)	total: 53.2s	remaining: 16.8s
415:	learn: 11.3884127	test: 7.1521718	best: 7.1507738 (405)	total: 53.4s	remaining: 16.7s
416:	learn: 11.3809363	test: 7.1523662	best: 7.1507738 (405)	total: 53.5s	remaining: 16.6s
4

498:	learn: 10.8789167	test: 7.1132925	best: 7.1132925 (498)	total: 1m 6s	remaining: 6.31s
499:	learn: 10.8739850	test: 7.1133676	best: 7.1132925 (498)	total: 1m 7s	remaining: 6.18s
500:	learn: 10.8692849	test: 7.1138571	best: 7.1132925 (498)	total: 1m 7s	remaining: 6.05s
501:	learn: 10.8654235	test: 7.1127796	best: 7.1127796 (501)	total: 1m 7s	remaining: 5.92s
502:	learn: 10.8614194	test: 7.1129405	best: 7.1127796 (501)	total: 1m 7s	remaining: 5.78s
503:	learn: 10.8574790	test: 7.1042780	best: 7.1042780 (503)	total: 1m 7s	remaining: 5.66s
504:	learn: 10.8532424	test: 7.1043423	best: 7.1042780 (503)	total: 1m 8s	remaining: 5.53s
505:	learn: 10.8480253	test: 7.1043631	best: 7.1042780 (503)	total: 1m 8s	remaining: 5.41s
506:	learn: 10.8420921	test: 7.1043022	best: 7.1042780 (503)	total: 1m 8s	remaining: 5.28s
507:	learn: 10.8373531	test: 7.0844142	best: 7.0844142 (507)	total: 1m 8s	remaining: 5.15s
508:	learn: 10.8325823	test: 7.0843276	best: 7.0843276 (508)	total: 1m 8s	remaining: 5.01s

In [90]:
pickle.dump(model_to_save, open('set_features_model_full_train.pkl', 'wb'))